import libraries(I provide all libs that I need when make this tasks, if you need some external import them here)

In [169]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import col
from pyspark.sql.functions import max, avg, min
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number,desc
from pyspark.sql.functions import when

create local SparkSession

In [170]:
spark=SparkSession.builder.appName("Spark_1").getOrCreate()

read csv with inferschema

In [171]:
df=spark.read.csv("./ds_salaries.csv",inferSchema=True,header=True)
df.show()

+---+---------+----------------+---------------+--------------------+--------+---------------+-------------+------------------+------------+----------------+------------+
|_c0|work_year|experience_level|employment_type|           job_title|  salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|
+---+---------+----------------+---------------+--------------------+--------+---------------+-------------+------------------+------------+----------------+------------+
|  0|     2020|              MI|             FT|      Data Scientist|   70000|            EUR|        79833|                DE|           0|              DE|           L|
|  1|     2020|              SE|             FT|Machine Learning ...|  260000|            USD|       260000|                JP|           0|              JP|           S|
|  2|     2020|              SE|             FT|   Big Data Engineer|   85000|            GBP|       109024|                GB|          50|     

read csv one more time with the same code and you will see that it almostly don't take time, because info already in SparkSession and it will not read nothing
from this file

In [172]:
df=spark.read.csv("./ds_salaries.csv",inferSchema=True,header=True)
df=df.withColumnRenamed("_c0","id")

write schema of scv on screen

In [173]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- work_year: integer (nullable = true)
 |-- experience_level: string (nullable = true)
 |-- employment_type: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- salary_currency: string (nullable = true)
 |-- salary_in_usd: integer (nullable = true)
 |-- employee_residence: string (nullable = true)
 |-- remote_ratio: integer (nullable = true)
 |-- company_location: string (nullable = true)
 |-- company_size: string (nullable = true)



create schema of this scv

In [174]:

#schema=df.schema
schema=StructType([StructField("_c0",IntegerType(),True),
                    StructField("work_year",IntegerType(),True),
                    StructField("experience_level",StringType(),True),
                    StructField("employment_type",StringType(),True),
                    StructField("job_title",StringType(),True),
                    StructField("salary", IntegerType(), True),
                    StructField("salary_currency", StringType(), True),
                    StructField("salary_in_usd", IntegerType(), True),
                    StructField("employee_residence", StringType(), True),
                    StructField("remote_ratio", IntegerType(), True),
                    StructField("company_location", StringType(), True),
                    StructField("company_size", StringType(), True)])

restart kernel without cleaning output and after restarting you need to initialize SparkSession, after initialize start execute only cells from cell with schema=
=StructType.... 
To restart kernel click Kernel, Restart.

read ds_salaries with predefined schema and compare results from this cell and cell with inferSchema

In [175]:
df_schema=spark.read.csv("ds_salaries.csv",header=True,schema=schema)
df_infer=spark.read.csv("ds_salaries.csv",header=True,inferSchema=True)
df_schema.printSchema
#df_infer.printSchema


<bound method DataFrame.printSchema of DataFrame[_c0: int, work_year: int, experience_level: string, employment_type: string, job_title: string, salary: int, salary_currency: string, salary_in_usd: int, employee_residence: string, remote_ratio: int, company_location: string, company_size: string]>

this happens because read operation is lazy(transformation), but if you use inferschema it start to be action that will create Spark Job, because Spark need to loop throw all file to check datatypes for all columns and this can harm to your code(if we compare to parquet, it will also go to check data types, but parquet provide meta information, so Spark will not go throw all file, he will just read meta information, but csv don't provide such meta information). Also header make Spark to create one more Spark Job to check first line
to define name of columns and remember to skeep it when reading. Actual reading start when you will use first action. More about Spark Jobs you will see in next topic

write schema of scv on screen one more time and compare with previous

In [176]:
df_infer.printSchema

<bound method DataFrame.printSchema of DataFrame[_c0: int, work_year: int, experience_level: string, employment_type: string, job_title: string, salary: int, salary_currency: string, salary_in_usd: int, employee_residence: string, remote_ratio: int, company_location: string, company_size: string]>

now continue to work with one of the dataframes that you create

print data in dataframe using df.show

In [177]:
df

DataFrame[id: int, work_year: int, experience_level: string, employment_type: string, job_title: string, salary: int, salary_currency: string, salary_in_usd: int, employee_residence: string, remote_ratio: int, company_location: string, company_size: string]

print data in dataframe using display(df.toPandas())

In [178]:
df.toPandas()

,id,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,0,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,0,DE,L
1,1,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,0,JP,S
2,2,2020,SE,FT,Big Data Engineer,85000,GBP,109024,GB,50,GB,M
3,3,2020,MI,FT,Product Data Analyst,20000,USD,20000,HN,0,HN,S
4,4,2020,SE,FT,Machine Learning Engineer,150000,USD,150000,US,50,US,L
...,...,...,...,...,...,...,...,...,...,...,...,...
602,602,2022,SE,FT,Data Engineer,154000,USD,154000,US,100,US,M
603,603,2022,SE,FT,Data Engineer,126000,USD,126000,US,100,US,M
604,604,2022,SE,FT,Data Analyst,129000,USD,129000,US,0,US,M
605,605,2022,SE,FT,Data Analyst,150000,USD,150000,US,100,US,M


create df_job_title that consists from all job_titles without duplicates

In [179]:
df_job_title=df.select("job_title").distinct()
df_job_title.show()

+--------------------+
|           job_title|
+--------------------+
|3D Computer Visio...|
|  Lead Data Engineer|
|Head of Machine L...|
|     Data Specialist|
| Data Analytics Lead|
|Machine Learning ...|
|   Lead Data Analyst|
|Data Engineering ...|
|Staff Data Scientist|
|       ETL Developer|
|Director of Data ...|
|Product Data Analyst|
|Principal Data Sc...|
|        AI Scientist|
|Director of Data ...|
|Machine Learning ...|
| Lead Data Scientist|
|Machine Learning ...|
|Data Science Engi...|
|Machine Learning ...|
+--------------------+
only showing top 20 rows



print all rows from df_job_titles without truncating jobs

In [180]:
df_job_title.show(truncate=False)

+----------------------------------------+
|job_title                               |
+----------------------------------------+
|3D Computer Vision Researcher           |
|Lead Data Engineer                      |
|Head of Machine Learning                |
|Data Specialist                         |
|Data Analytics Lead                     |
|Machine Learning Scientist              |
|Lead Data Analyst                       |
|Data Engineering Manager                |
|Staff Data Scientist                    |
|ETL Developer                           |
|Director of Data Engineering            |
|Product Data Analyst                    |
|Principal Data Scientist                |
|AI Scientist                            |
|Director of Data Science                |
|Machine Learning Engineer               |
|Lead Data Scientist                     |
|Machine Learning Infrastructure Engineer|
|Data Science Engineer                   |
|Machine Learning Manager                |
+----------

create  df_analytic that will consists from max, avg, min USD salaries for all job_titles using groupBy. name of fields is avg_salary, min_salary, max_salary

In [181]:
# df_analytic=df.filter(df["salary_currency"]=='USD').groupBy("job_title".agg((avg("salary").alias("avg_salary"),
#          min("salary").alias("min_salary"),
#          max("salary").alias("max_salary"))))
# df_analytic.show()
df_analytic = df.filter(df["salary_currency"]=='USD').groupBy("job_title").agg(
         avg("salary_in_usd").alias("avg_salary"),
         min("salary_in_usd").alias("min_salary"),
         max("salary_in_usd").alias("max_salary"))
df_analytic.show()

+--------------------+------------------+----------+----------+
|           job_title|        avg_salary|min_salary|max_salary|
+--------------------+------------------+----------+----------+
|  Lead Data Engineer|          154250.0|     56000|    276000|
|     Data Specialist|          165000.0|    165000|    165000|
| Data Analytics Lead|          405000.0|    405000|    405000|
|Machine Learning ...|          158412.5|     12000|    260000|
|   Lead Data Analyst|          128500.0|     87000|    170000|
|Data Engineering ...|          159000.0|    150000|    174000|
|Staff Data Scientist|          105000.0|    105000|    105000|
|Director of Data ...|          200000.0|    200000|    200000|
|Product Data Analyst|           20000.0|     20000|     20000|
|Principal Data Sc...|          255500.0|    151000|    416000|
|        AI Scientist|           79800.0|     12000|    200000|
|Director of Data ...|247666.66666666666|    168000|    325000|
|Machine Learning ...|          140488.0

print all rows from df_analytic without trancating jobs

In [182]:
df_analytic.show(truncate=True)

+--------------------+------------------+----------+----------+
|           job_title|        avg_salary|min_salary|max_salary|
+--------------------+------------------+----------+----------+
|  Lead Data Engineer|          154250.0|     56000|    276000|
|     Data Specialist|          165000.0|    165000|    165000|
| Data Analytics Lead|          405000.0|    405000|    405000|
|Machine Learning ...|          158412.5|     12000|    260000|
|   Lead Data Analyst|          128500.0|     87000|    170000|
|Data Engineering ...|          159000.0|    150000|    174000|
|Staff Data Scientist|          105000.0|    105000|    105000|
|Director of Data ...|          200000.0|    200000|    200000|
|Product Data Analyst|           20000.0|     20000|     20000|
|Principal Data Sc...|          255500.0|    151000|    416000|
|        AI Scientist|           79800.0|     12000|    200000|
|Director of Data ...|247666.66666666666|    168000|    325000|
|Machine Learning ...|          140488.0

now you need to add in df_analytic column row_id, that will show order of all job_titles depending on avg salary. they should be descending

In [183]:
df_analytic=df_analytic.withColumn("row_id",row_number().over(Window.orderBy(desc("avg_salary"))
))
df_analytic.show()


+--------------------+------------------+----------+----------+------+
|           job_title|        avg_salary|min_salary|max_salary|row_id|
+--------------------+------------------+----------+----------+------+
| Data Analytics Lead|          405000.0|    405000|    405000|     1|
|Principal Data En...| 328333.3333333333|    185000|    600000|     2|
|Financial Data An...|          275000.0|    100000|    450000|     3|
|         ML Engineer|          263000.0|    256000|    270000|     4|
|Principal Data Sc...|          255500.0|    151000|    416000|     5|
|Director of Data ...|247666.66666666666|    168000|    325000|     6|
|Applied Data Scie...|          238000.0|    157000|    380000|     7|
|        Head of Data|221666.66666666666|    200000|    235000|     8|
|Director of Data ...|          200000.0|    200000|    200000|     9|
|Machine Learning ...|          195000.0|    195000|    195000|    10|
|Applied Machine L...|          178800.0|     38400|    423000|    11|
|     

print all data from df_analytic

In [184]:
df_analytic.show()

+--------------------+------------------+----------+----------+------+
|           job_title|        avg_salary|min_salary|max_salary|row_id|
+--------------------+------------------+----------+----------+------+
| Data Analytics Lead|          405000.0|    405000|    405000|     1|
|Principal Data En...| 328333.3333333333|    185000|    600000|     2|
|Financial Data An...|          275000.0|    100000|    450000|     3|
|         ML Engineer|          263000.0|    256000|    270000|     4|
|Principal Data Sc...|          255500.0|    151000|    416000|     5|
|Director of Data ...|247666.66666666666|    168000|    325000|     6|
|Applied Data Scie...|          238000.0|    157000|    380000|     7|
|        Head of Data|221666.66666666666|    200000|    235000|     8|
|Director of Data ...|          200000.0|    200000|    200000|     9|
|Machine Learning ...|          195000.0|    195000|    195000|    10|
|Applied Machine L...|          178800.0|     38400|    423000|    11|
|     

it isn't beautifull, so we need to put now row_id on first place in df_analytic

In [185]:
#df_analytic=df_analytic.select("row_id",*df_analytic.columns)
columns=df_analytic.columns
columns.remove("row_id")
columns.insert(0,"row_id")
df_analytic.select(*columns)

DataFrame[row_id: int, job_title: string, avg_salary: double, min_salary: int, max_salary: int]

print df_analytic now

In [186]:
df_analytic.show()

+--------------------+------------------+----------+----------+------+
|           job_title|        avg_salary|min_salary|max_salary|row_id|
+--------------------+------------------+----------+----------+------+
| Data Analytics Lead|          405000.0|    405000|    405000|     1|
|Principal Data En...| 328333.3333333333|    185000|    600000|     2|
|Financial Data An...|          275000.0|    100000|    450000|     3|
|         ML Engineer|          263000.0|    256000|    270000|     4|
|Principal Data Sc...|          255500.0|    151000|    416000|     5|
|Director of Data ...|247666.66666666666|    168000|    325000|     6|
|Applied Data Scie...|          238000.0|    157000|    380000|     7|
|        Head of Data|221666.66666666666|    200000|    235000|     8|
|Director of Data ...|          200000.0|    200000|    200000|     9|
|Machine Learning ...|          195000.0|    195000|    195000|    10|
|Applied Machine L...|          178800.0|     38400|    423000|    11|
|     

here you need to create df_exp_lvl with the biggest usd_salary(biggest_salary) for each experience_level(you need to save all fields like in entire dataframe)

In [190]:
df_usd_salaries = df.filter(df["salary_currency"] == "USD")
windowSpec = Window.partitionBy("experience_level").orderBy(df_usd_salaries["salary_in_usd"].desc())
df_exp_lvl = df_usd_salaries.withColumn("biggest_salary", max("salary_in_usd").over(windowSpec))

+---+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+--------------+
| id|work_year|experience_level|employment_type|           job_title|salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|biggest_salary|
+---+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+--------------+
| 37|     2020|              EN|             FT|Machine Learning ...|250000|            USD|       250000|                US|          50|              US|           L|        250000|
|115|     2021|              EN|             FT|Machine Learning ...|225000|            USD|       225000|                US|         100|              US|           L|        250000|
|510|     2022|              EN|             FT|Computer Vision S...|150000|    

print here df_exp_lvl

In [191]:
df_exp_lvl.show()

+---+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+--------------+
| id|work_year|experience_level|employment_type|           job_title|salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|biggest_salary|
+---+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+--------------+
| 37|     2020|              EN|             FT|Machine Learning ...|250000|            USD|       250000|                US|          50|              US|           L|        250000|
|115|     2021|              EN|             FT|Machine Learning ...|225000|            USD|       225000|                US|         100|              US|           L|        250000|
|510|     2022|              EN|             FT|Computer Vision S...|150000|    

create df_best that consists from rows where salary of guy same as biggest salary for other people in his exp_lvl and choose only columns: id, experience_level, biggest_salary, employee_residence

In [192]:
df_best = df.withColumn("biggest_salary", max("salary_in_usd").over(Window.partitionBy("experience_level").orderBy(df["salary_in_usd"].desc()))).filter(col("salary_in_usd") == col("biggest_salary"))
df_best=df_best.select("id",("experience_level"),col("biggest_salary"),col("employee_residence"))


print df_best

In [193]:
df_best.show()

+---+----------------+--------------+------------------+
| id|experience_level|biggest_salary|employee_residence|
+---+----------------+--------------+------------------+
| 37|              EN|        250000|                US|
|252|              EX|        600000|                US|
| 33|              MI|        450000|                US|
| 97|              MI|        450000|                US|
| 63|              SE|        412000|                US|
+---+----------------+--------------+------------------+



drop duplicates if exist by experience_level

In [148]:
df_best = df_best.dropDuplicates(['experience_level'])

print df_best

In [149]:
df_best.show()

+---+----------------+--------------+------------------+
| id|experience_level|biggest_salary|employee_residence|
+---+----------------+--------------+------------------+
| 37|              EN|        250000|                US|
|252|              EX|        600000|                US|
| 33|              MI|        450000|                US|
| 63|              SE|        412000|                US|
+---+----------------+--------------+------------------+



create df_new_best from df_best without id, and make the next: when exp_level = MI we want middle, when SE we want senior, else Null

In [155]:
df_new_best=df_best.select(df_best.columns[1:3])
df_new_best = df_new_best.withColumn("level", when(df_new_best["experience_level"] == "MI", "middle").when(df_new_best["experience_level"] == "SE", "senior").otherwise(None))

print df_new_best

In [156]:
df_new_best.show()

+----------------+--------------+------+
|experience_level|biggest_salary| level|
+----------------+--------------+------+
|              EN|        250000|  NULL|
|              EX|        600000|  NULL|
|              MI|        450000|middle|
|              SE|        412000|senior|
+----------------+--------------+------+



write df_new_best like 1.csv and load then it to df_final

In [198]:
#df_new_best.write.format("csv").mode('overwrite').save("1.csv")
#df_final=spark.read.csv("./1.csv",inferSchema=True,header=True)
df_final=df_new_best

print df_final

In [199]:
df_final.show()

+----------------+--------------+------+
|experience_level|biggest_salary| level|
+----------------+--------------+------+
|              EN|        250000|  NULL|
|              EX|        600000|  NULL|
|              MI|        450000|middle|
|              SE|        412000|senior|
+----------------+--------------+------+



filter df_final to delete experience_level where it Null, then join this table by biggest_salary(salary_in_usd) and employee_residence with entire df

In [212]:
df_final=df_final.filter(df_final["experience_level"].isNotNull())

df_final = df.join(df_exp_lvl, ["salary_in_usd"], "inner")


+----------------+--------------+------+
|experience_level|biggest_salary| level|
+----------------+--------------+------+
|              MI|        450000|middle|
|              SE|        412000|senior|
+----------------+--------------+------+



print df_final

In [213]:
df_final.show()

+-------------+---+---------+----------------+---------------+--------------------+------+---------------+------------------+------------+----------------+------------+---+---------+----------------+---------------+--------------------+------+---------------+------------------+------------+----------------+------------+--------------+
|salary_in_usd| id|work_year|experience_level|employment_type|           job_title|salary|salary_currency|employee_residence|remote_ratio|company_location|company_size| id|work_year|experience_level|employment_type|           job_title|salary|salary_currency|employee_residence|remote_ratio|company_location|company_size|biggest_salary|
+-------------+---+---------+----------------+---------------+--------------------+------+---------------+------------------+------------+----------------+------------+---+---------+----------------+---------------+--------------------+------+---------------+------------------+------------+----------------+------------+-----

last task is to save in variable and then print this variable of the biggest salary_in_usd from df_final

In [224]:
max_salary=df_final.agg({"salary_in_usd":"max"}).collect()[0][0]
print(max_salary)

600000


It is the end of PySpark basics. In other lessons you will learn optimizations technics and how to make distributed system